[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mbertani/conformal-prediction/blob/main/02_Conformal_Prediction_NLP.ipynb)

The purpose of this notebook is to show how Conformal Prediction can be used to better filter toxic or hateful text via conformal outlier detection. We will only use the non-toxic data, and then with type-1 error control identify the toxic outliers

This notebook uses Kennedy et al.'s (2020) [pre-trained RoBERTa-Base](https://huggingface.co/ucberkeley-dlab/hate-measure-roberta-base) to score hate speech provided by [publicly released dataset](https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech) described in Kennedy et al. (2020) and Sachdeva et al. (2022).

This notebook is inspired by Angelopoulos and Bates' (2021) [A gentle introduction to conformal prediction and distribution-free uncertainty quantification](https://github.com/aangelopoulos/conformal-prediction/blob/main/notebooks/toxic-text-outlier-detection.ipynb)


References:\
Angelopoulos, A. N., & Bates, S. (2021). A gentle introduction to conformal prediction and distribution-free uncertainty quantification. arXiv preprint arXiv:2107.07511.\
Kennedy, C. J., Bacon, G., Sahn, A., & von Vacano, C. (2020). Constructing interval variables via faceted Rasch measurement and multitask deep learning: a hate speech application. arXiv preprint arXiv:2009.10277.\
Pratik Sachdeva, Renata Barreto, Geoff Bacon, Alexander Sahn, Claudia von Vacano, and Chris Kennedy. 2022. The Measuring Hate Speech Corpus: Leveraging Rasch Measurement Theory for Data Perspectivism. In Proceedings of the 1st Workshop on Perspectivist Approaches to NLP @LREC2022, pages 83–94, Marseille, France. European Language Resources Association.

# Download inference data

In [1]:
!mkdir assets
!curl -o \
    assets/NLP_y_pred_n_data_samples=5000.pickle \
    https://raw.githubusercontent.com/mbertani/conformal-prediction/refs/heads/main/assets/NLP_y_pred_n_data_samples%3D5000.pickle

mkdir: assets: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20129  100 20129    0     0  55750      0 --:--:-- --:--:-- --:--:-- 55759


# Libraries

In [ ]:
!pip install jupyter
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface_hub
!pip install -q torch
!pip install tf-keras

# Imports & Verbosities

In [2]:
import numpy as np
import datasets, transformers
from huggingface_hub import from_pretrained_keras
import tensorflow as tf
import pickle
import os

transformers.logging.set_verbosity_error()
tf.get_logger().setLevel('ERROR')

# Globals

In [3]:
n_data_samples = 5000 # download this number of data samples from https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech

alpha = 0.1 # 1-alpha is the desired type-1 error (10%). If the desired level is set to 10%, the conformal predictor will be adjusted to ensure that the probability of falsely identifying non-toxic speech as toxic does not exceed 10%
n = 200 # n calibration points

model_path = "ucberkeley-dlab/hate-measure-roberta-base" #https://huggingface.co/ucberkeley-dlab/hate-measure-roberta-base
tokenizer_path = "roberta-base"

data_path = 'ucberkeley-dlab/measuring-hate-speech' #https://huggingface.co/FacebookAI/roberta-base

model_inference_results_cached_path = f"assets/NLP_y_pred_n_data_samples={n_data_samples}.pickle"

# Model & Tokenizer

In [4]:
model = from_pretrained_keras(model_path, verbose=0)
model.compile()

tokenizer = transformers.RobertaTokenizer.from_pretrained(tokenizer_path)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

# Data & Tokenization

In [7]:
sub_df = datasets.load_dataset(data_path, 'default', split='train').to_pandas()[['text', 'hate_speech_score']].sample(n=n_data_samples)

tokens = tokenizer(sub_df['text'].values.tolist(), return_tensors='np', padding="max_length", max_length=247, truncation=True)

# Inference

In [8]:
if os.path.exists(model_inference_results_cached_path):
    with open(model_inference_results_cached_path, 'rb') as handle:
        y_pred = pickle.load(handle)
else:
    y_pred = model.predict([tokens['input_ids'], tokens['attention_mask']])
    with open(model_inference_results_cached_path, 'wb') as handle:
        pickle.dump(y_pred, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
# with open(f"assets/NLP_y_pred_n_data_samples={n_data_samples}.pickle", 'wb') as handle:
#     pickle.dump(y_pred, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
# with open(f"assets/NLP_y_pred_n_data_samples={n_data_samples}.pickle", 'rb') as handle:
#     y_pred = pickle.load(handle)

array([[ 2.5247893 ],
       [-0.88125783],
       [ 0.97084135],
       ...,
       [ 2.1258526 ],
       [-2.5380704 ],
       [ 1.1036235 ]], dtype=float32)

# Normalise data

In [9]:
# Continuous data is in the range = [-8.34 - 6.3]
preds = np.array([(float(x)+8.34)/(6.3+8.34) for x in y_pred.flatten().tolist()])
is_toxic = np.array([round((float(x)+8.34)/(6.3+8.34)) for x in sub_df['hate_speech_score'].values.tolist()])

In [10]:
# Look at only the non-toxic data
nontoxic = is_toxic == 0

preds_nontoxic = preds[nontoxic]
preds_toxic = preds[np.invert(nontoxic)]

In [11]:
# Split nontoxic data into calibration and validation sets
# The conformal predictor can be calibrated to control the type-1 error rate to a desired level, ensuring that the probability of missing toxic speech is kept below a threshold α
idx = np.array([1] * n + [0] * (preds_nontoxic.shape[0]-n)) > 0
cal_scores, val_scores = preds_nontoxic[idx], preds_nontoxic[np.invert(idx)]

### Conformal outlier detection happens here

In [12]:
# Use the outlier detection method to get a threshold on the toxicities
# qhat ​is a quantile value derived from the nonconformity scores of the calibration set
qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n)

# Perform outlier detection on the ind and ood data
outlier_ind = val_scores > qhat # Identifying outliers in the in-distribution data. The goal is to have the proportion of outliers be no more than alpha on average
outlier_ood = preds_toxic > qhat # Identifying outliers in the out-of-distribution data. Out-of-distribution data is inherently more unpredictable and may not conform to the same patterns as the in-distribution data


In [13]:
# Calculate type-1 and type-2 errors
# Type-1 Error (False Negative) - true label (toxic or non-toxic) is not included in the predicted confidence region.
# Type-2 Error (False Positive) - predicted confidence region includes both toxic and non-toxic labels, even when the true label is non-toxic.

# There's a trade-off between the two types of errors. Reducing type-1 errors (ensuring all toxic speech is identified) might increase type-2 errors (more non-toxic speech being flagged as potentially toxic)

# The conformal predictor can be calibrated to control the type-1 error rate to a desired level, ensuring that the probability of missing toxic speech is kept below a threshold, say α. However, this often results in larger prediction sets, which could lead to higher type-2 errors.
type1 = outlier_ind.mean()
type2 = 1-outlier_ood.mean()
print(f"The type-1 error is {type1:.4f}, the type-2 error is {type2:.4f}, and the threshold is {qhat:.4f}.")

The type-1 error is 0.0476, the type-2 error is 0.9655, and the threshold is 0.8409.


In [14]:
# # Show predictions made by the model that are outliers
predictions_post_cf = sub_df['text'].values[nontoxic][n:][outlier_ind==True].tolist()

_=[print(item) for item in list(np.random.choice(predictions_post_cf, size=(10,)))]

Men think that they can get away with this sort of behaviour when they pick on women. In this case, apart from the fact that I'm surprised he can actually string a coherent sentence together, there's now an opportunity to identify & charge him @anitasethi URL
>Some incels give a lot but get nothing back   And so should be rewarded with pussy? Are you aware that said pussy is attached to a fully realized human being with the exact same natural rights to happiness as you? And that she does not deserve to be forced to have sex with anyone she finds undesirable?    Do you get that women have the same right to reject men that men have to reject women?    How committed are you to the belief that women are holes, rather than human beings deserving of respect?
I agree ISIS is true and needs to be stopped!
No, the chances of a muslim shooting you in America is almost nil. There are over 50K gun deaths every year christian USA. We should be focusing our attention on Christians and less on peace 